In [22]:
import geopandas as gp
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 210)

In [113]:
cpuc_zipfile    = "zip:///Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/Fixed_Consumer_Deployment_2020.zip"
census_excel    = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/Census_Blocks_2010.xlsx"
bc_csv          = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/BC_collapsed.csv"
telco_prof_csv  = "C:/Users/jongo/Documents/College/Class/GRIDS/Microtelcos/data/telco_profiles.csv"

#cpuc = gp.read_file(cpuc_zipfile)
#cpuc_simple = cpuc[['BlockCode', 'DBA', 'FRN','TechCode']]
#cpuc_simple['BlockCode']  = cpuc_simple.BlockCode.astype('int64')

#census = pd.read_excel(census_excel)
#census = census.rename(columns = {"SE_T002_001":"Population", "SE_T002_002":"Population Density","SE_T002_006":"Area (sq mi)", "SE_T058_001":"Households"})

bc = pd.read_csv(bc_csv)
bc = bc.rename(columns = {"blockcode":"BlockCode", "blockgroup":"BlockGroup"})

#telco_profiles = pd.read_csv(telco_prof_csv)

In [117]:
# Assign 0-1 Encoding for each ISP size based on household

telco_profiles = pd.read_csv(telco_prof_csv)

telco_profiles = telco_profiles.fillna(0)

telco_profiles['telco_under_2000'] = 0
telco_profiles['telco_2000_4999']  = 0
telco_profiles['telco_5000_9999']  = 0
telco_profiles['telco_over_10000'] = 0

for DBA in telco_profiles['DBA'].index:

    if telco_profiles.loc[DBA,'Households'] < 2000:
        telco_profiles.loc[DBA, 'telco_under_2000'] = 1

    elif telco_profiles.loc[DBA,'Households'] < 5000:
        telco_profiles.loc[DBA, 'telco_2000_4999']  = 1

    elif telco_profiles.loc[DBA,'Households'] < 10000:
        telco_profiles.loc[DBA, 'telco_5000_9999']  = 1

    else:
        telco_profiles.loc[DBA, 'telco_over_10000'] = 1

telco_profiles = telco_profiles.drop(['Population','Area (sq mi)','Households','Business_only','web'], axis = 1)

# Uncomment below to save telco_profiles
#telco_profiles.to_csv('telco_profiles_1.csv')

In [110]:
telco_profiles

,DBA,Small ILEC,CHCF_A,local,coop,private,muni,telco_under_2000,telco_2000_4999,telco_5000_9999,telco_over_10000
0,"AT&T Service, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
1,Charter Communications Inc,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
2,Comcast,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
3,Frontier Communications,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
4,Cox Communications,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
5,Sonic.net,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
6,Wave Broadband,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
7,Consolidated Communications,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
8,Suddenlink Communications,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
9,"DigitalPath, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1


In [111]:
## Join cpuc_simple and census
joined_table = pd.merge(cpuc_simple, census, on='BlockCode', how='outer').drop_duplicates()

#grouped_table_by_tech = joined_table.groupby(['TechCode','DBA']).sum()
#grouped_table_by_tech = grouped_table_by_tech[['Population', 'Area (sq mi)', 'Households']]


# remove the TechCode column from the joined_table
#joined_table = joined_table[['DBA', 'FRN', 'Population','Population Density', 'Area (sq mi)', 'Households']].drop_duplicates()

grouped_table = joined_table.groupby(['DBA']).sum()
grouped_table = grouped_table[['Population', 'Area (sq mi)', 'Households']]

In [112]:
joined_table.dropna()

,BlockCode,DBA,FRN,Population,Population Density,Area (sq mi),Households
0,60230101023137,Aan Chuuphan,0000830321,3,468.46520,0.006404,2
1,60230101023139,Aan Chuuphan,0000830321,0,0.00000,0.013804,0
2,60230101023141,Aan Chuuphan,0000830321,0,0.00000,0.005308,0
3,60230101023154,Aan Chuuphan,0000830321,18,54.22746,0.331935,10
4,60230101023155,Aan Chuuphan,0000830321,0,0.00000,2.354610,0
...,...,...,...,...,...,...,...
1422554,61150411004062,NaN,NaN,12,104.79060,0.114514,6
1422555,61150411004063,NaN,NaN,0,0.00000,0.032129,0
1422556,61150411004064,NaN,NaN,2,2156.52600,0.000927,1
1422557,61150411004065,NaN,NaN,3,1545.33900,0.001941,2


In [ ]:
grouped_table_by_tech

In [ ]:
grouped_table.to_csv("telecoms_population_households_2010.csv")
grouped_table_by_tech.to_csv("telecoms_population_households_2010_by_tech.csv")

In [118]:
#Join cpuc_simple and census, then merge with the telco_profiles file, then merge with the bc file

telco_cols  = ['COLR','Small ILEC', 'CHCF_A','local','coop','private','muni',
               'telco_under_2000','telco_2000_4999','telco_5000_9999','telco_over_10000']

# Remove Tech Code from cpuc_simple
cpuc_simple = cpuc[['BlockCode', 'DBA', 'FRN']]
cpuc_simple['BlockCode']  = cpuc_simple.BlockCode.astype('int64')

joined_table_1 = pd.merge(cpuc_simple, bc, on='BlockCode', how='outer').drop_duplicates()
joined_table_1 = joined_table_1.dropna(axis=0, subset=['DBA'])

joined_table_2 = pd.merge(joined_table_1,telco_profiles[telco_cols], on='DBA', how = 'outer').drop_duplicates()

joined_table_2 = joined_table_2.dropna(axis=0, subset=['BlockCode'])
joined_table_2['BlockCode'] = joined_table_2.BlockCode.astype('int64')
joined_table_2 = joined_table_2.set_index('BlockCode')
#joined_table_2 = joined_table_2[telco_cols]

# joined_table_3 = pd.merge(joined_table_2, bc[['BlockCode','BlockGroup','LAcounty','n_blocks','n_isp','n_bb_25','n_bb_100','n_fib','n_cab','n_dsl',
#                                              'n_fixed_wireless','max_up','max_dn','BGpop','shrpop10','served','served25',
#                                              'served100','served_fib','comp','comp25','comp100','comp_fib']],
#                           on = 'BlockCode', how = 'outer').drop_duplicates()

<ipython-input-118-fbd6243eb7b8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpuc_simple['BlockCode']  = cpuc_simple.BlockCode.astype('int64')


KeyError: 'DBA'

In [119]:
joined_table_1

,BlockCode,DBA,FRN,BlockGroup,POP10,PopDensity,LAcounty,n_blocks,n_isp,n_bb_25,...,BGpop,shrpop10,served,served25,served100,served_fib,comp,comp25,comp100,comp_fib
0,60230101023137,Aan Chuuphan,0000830321,60230101023,3,468.465210,0,272,1,0,...,612,0.004902,1,0,0,0,0.0,NaN,NaN,NaN
1,60230101023139,Aan Chuuphan,0000830321,60230101023,0,0.000000,0,272,1,0,...,612,0.000000,1,0,0,0,0.0,NaN,NaN,NaN
2,60230101023141,Aan Chuuphan,0000830321,60230101023,0,0.000000,0,272,1,0,...,612,0.000000,1,0,0,0,0.0,NaN,NaN,NaN
3,60230101023154,Aan Chuuphan,0000830321,60230101023,18,54.227463,0,272,1,0,...,612,0.029412,1,0,0,0,0.0,NaN,NaN,NaN
4,60230101023155,Aan Chuuphan,0000830321,60230101023,0,0.000000,0,272,1,0,...,612,0.000000,1,0,0,0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094415,60910100002013,PlumasSierra,0022537807,60910100002,2,1241.605000,0,100,1,0,...,711,0.002813,1,0,0,0,0.0,NaN,NaN,NaN
1094416,60910100002098,PlumasSierra,0022537807,60910100002,2,1146.012300,0,100,1,0,...,711,0.002813,1,0,0,0,0.0,NaN,NaN,NaN
1094417,60910100003065,PlumasSierra,0022537807,60910100003,24,139.717090,0,475,1,0,...,563,0.042629,1,0,0,0,0.0,NaN,NaN,NaN
1094418,60910100004001,PlumasSierra,0022537807,60910100004,6,3.031815,0,597,1,0,...,686,0.008746,1,0,0,0,0.0,NaN,NaN,NaN


In [6]:
del joined_table
del joined_table_1
del grouped_table
del grouped_table_by_tech
del cpuc
del cpuc_simple
del census
del telco_profiles

In [120]:
telco_cols     = ["Small ILEC", 'CHCF_A','local','coop','private','muni',
                  'telco_under_2000','telco_2000_4999','telco_5000_9999','telco_over_10000']

# Set index to BlockCode for both bc and joined_table_2 for fast access
bc_1 = bc
bc_1['BlockCode'] = bc_1.BlockCode.astype('int64')
bc_1 = bc_1.set_index('BlockCode')

for col in telco_cols:
    bc_1[col] = 0

bc_1 = bc_1[telco_cols]

# Find which blocks are served by each kind of ISP
for block_code in bc_1.index:
    temp = joined_table_2.loc[[block_code]]
    
    if len(temp)==1:
        temp = temp.to_frame().T.max()
        bc_1.loc[block_code,telco_cols] = temp
    else:
        temp = temp[telco_cols].max()
        bc_1.loc[block_code,telco_cols] = temp
    
    del temp
    
    print(block_code)

bc_1.to_csv("bc_jag.csv")

60014001001026


AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [90]:
temp = joined_table_2.loc[60014001001028]
print(temp)
temp.max()

bc_1.loc[60014001001028, telco_cols] = temp
bc_1.loc[60014001001028, telco_cols]

Small ILEC          0.0
CHCF_A              0.0
local               0.0
coop                0.0
private             0.0
muni                0.0
telco_under_2000    0.0
telco_2000_4999     0.0
telco_5000_9999     0.0
telco_over_10000    1.0
Name: 60014001001028, dtype: float64


Small ILEC          0.0
CHCF_A              0.0
local               0.0
coop                0.0
private             0.0
muni                0.0
telco_under_2000    0.0
telco_2000_4999     0.0
telco_5000_9999     0.0
telco_over_10000    1.0
Name: 60014001001028, dtype: float64

In [91]:
#bc_1.head(100)
joined_table_2

,Small ILEC,CHCF_A,local,coop,private,muni,telco_under_2000,telco_2000_4999,telco_5000_9999,telco_over_10000
BlockCode,,,,,,,,,,
60230101023137,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
60230101023139,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
60230101023141,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
60230101023154,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
60230101023155,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
61150411004062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61150411004063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61150411004064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create the block group aggregate dataframe

bc_groups_cols = ["BGpop","LAcounty",
                  "served","served25","served100","served_fib","served_un","served_under25","served_under100",
                  "comp","comp25","comp100","comp_fib","comp_un","comp_under25","comp_under100", 
                  "max_up","max_dn","avg_max_up","avg_max_dn"]

bc_groups = pd.DataFrame(columns=bc_groups_cols, index = joined_table_3['BlockGroup'].unique())

# initialize the bc columns to 0
for col in telco_cols:
    bc[col] = 0

# Find which blocks are served by each kind of ISP
for block_code in bc['BlockCode']:
    cond = (joined_table_2['BlockCode']==block_code)
    temp = joined_table_2.loc[cond]
    
    for col in telco_cols:
        bc[col].loc[bc['BlockCode']==block_code] = temp[col].max()
        
    print(block_code)

count = 1
        
# Group each block together
for group in bc_groups.index:
    
    print(1, ' / ', len(bc_groups.index))
    
    if pd.isnull(group) is None:
        continue
    
    cond = (bc['BlockGroup']==group)
    temp = bc.loc[cond]
    
    n_blocks = len(temp)
    #n_isps   = len(temp['DBA'].unique())
    
    bc_groups["BGpop"].loc[group]           = temp["BGpop"].iloc[0]
    bc_groups["LAcounty"].loc[group]        = temp["LAcounty"].iloc[0]
    
    # Information about service coverage in general
    bc_groups["served"].loc[group]          = (temp["served"]*temp['shrpop10']).sum()
    bc_groups["served25"].loc[group]        = (temp["served25"]*temp['shrpop10']).sum()
    bc_groups["served100"].loc[group]       = (temp["served100"]*temp['shrpop10']).sum()
    bc_groups["served_fib"].loc[group]      = (temp["served_fib"]*temp['shrpop10']).sum()
    bc_groups["served_un"].loc[group]       = 1.0 - bc_groups["served"].loc[group]
    bc_groups["served_under25"].loc[group]  = bc_groups["served"].loc[group] - bc_groups["served25"].loc[group]
    bc_groups["served_under100"].loc[group] = bc_groups["served"].loc[group] - bc_groups["served100"].loc[group]
    
    bc_groups["comp"].loc[group]            = (temp["comp"]*temp['shrpop10']).sum()
    bc_groups["comp25"].loc[group]          = (temp["comp25"]*temp['shrpop10']).sum()
    bc_groups["comp100"].loc[group]         = (temp["comp100"]*temp['shrpop10']).sum()
    bc_groups["comp_fib"].loc[group]        = (temp["comp_fib"]*temp['shrpop10']).sum()
    bc_groups["comp_un"].loc[group]         = 1.0 - bc_groups["comp"].loc[group]
    bc_groups["comp_under25"].loc[group]    = bc_groups["comp"].loc[group] - bc_groups["comp25"].loc[group]
    bc_groups["comp_under100"].loc[group]   = bc_groups["comp"].loc[group] - bc_groups["comp100"].loc[group]
    
    # absolute maximum and average maximum speeds for each block group
    bc_groups["max_up"].loc[group]          = temp["max_up"].max()
    bc_groups["max_dn"].loc[group]          = temp["max_dn"].max()
    bc_groups["avg_max_up"].loc[group]      = (temp["max_up"]*temp['shrpop10']).sum()
    bc_groups["avg_max_dn"].loc[group]      = (temp["max_dn"]*temp['shrpop10']).sum()
    
    # information about small ISP service coverage
    bc_groups["Small ILEC"].loc[group]      = (temp["Small ILEC"]*temp['shrpop10']).sum()
    bc_groups["CHCF_A"].loc[group]          = (temp["CHCF_A"]*temp['shrpop10']).sum()
    bc_groups["local"].loc[group]           = (temp["local"]*temp['shrpop10']).sum()
    bc_groups["coop"].loc[group]            = (temp["coop"]*temp['shrpop10']).sum()
    bc_groups["private"].loc[group]         = (temp["private"]*temp['shrpop10']).sum()
    bc_groups["muni"].loc[group]            = (temp["muni"]*temp['shrpop10']).sum()
    
    # percentage of people reached by isps of each size
    bc_groups["telco_under_2000"].loc[group] = (temp["telco_under_2000"]*temp['shrpop10']).sum()
    bc_groups["telco_2000_4999"].loc[group]  = (temp["telco_2000_4999"]*temp['shrpop10']).sum()
    bc_groups["telco_5000_9999"].loc[group]  = (temp["telco_5000_9999"]*temp['shrpop10']).sum()
    bc_groups["telco_over_10000"].loc[group] = (temp["telco_over_10000"]*temp['shrpop10']).sum()
    
    #print('original temp: ',len(temp))
    # Remove the ISP-specific information
    #temp = temp.drop(['DBA','FRN'], axis=1)
    temp = temp.drop_duplicates()
    print('new temp: ',len(temp))

In [ ]:
bc

In [ ]:
temp.columns

In [ ]:
# convert all very small numbers to 0
bc_groups[bc_groups < 0.0000001] = 0

# Save the bc_groups to csv
bc_groups.to_csv("BC_collapsed_by_group_1.csv")